In [31]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [32]:
import pandas as pd
import numpy as np
import os

In [ ]:
torch.manual_seed(1)

In [33]:
df_labelled = pd.read_csv('./YAD/Labelled+webmini.csv')
print(df_labelled)
print(type(df_labelled))

     Unnamed: 0        ID  Gender  Age  PHQ9_total  GAD7_total STAI_X1_total  \
0             0  YAD30211       1   27          14           9            47   
1             1  YAD30504       1   24           3           3            33   
2             2  YAD30211       1   26          22          12            47   
3             3  YAD30212       1   24          22          10            29   
4             4  YAD30213       1   28           8           3            36   
..          ...       ...     ...  ...         ...         ...           ...   
289         289  MDD20019       1   24          10           8            64   
290         290  MDD20020       0   25          10          14            71   
291         291  MDD20021       0   31          17          12            62   
292         292  MDD20022       0   20          14          11         empty   
293         293  MDD20023       1   25           4           0            36   

    LSAS_performance_fear LSAS_situatio

In [34]:
df_labelled['STAI_X1_total'] = pd.to_numeric(df_labelled['STAI_X1_total'], errors='coerce') 

print(df_labelled['STAI_X1_total'].dtypes)

float64


In [35]:
train_df_labelled = df_labelled[df_labelled['site'].isin(['KAIST', 'Gachon'])]
test_df_labelled = df_labelled[df_labelled['site'] == 'SMC']
valid_df_labelled = df_labelled[df_labelled['site'] == 'SNU']

In [36]:
train_x = train_df_labelled.loc[:, ['PHQ9_total', 'GAD7_total', 'STAI_X1_total']]
test_x = test_df_labelled.loc[:, ['PHQ9_total', 'GAD7_total', 'STAI_X1_total']]
valid_x = valid_df_labelled.loc[:, ['PHQ9_total', 'GAD7_total', 'STAI_X1_total']]

print(train_x)

     PHQ9_total  GAD7_total  STAI_X1_total
0            14           9           47.0
1             3           3           33.0
2            22          12           47.0
3            22          10           29.0
4             8           3           36.0
..          ...         ...            ...
134           2           5           45.0
135           9           2           29.0
136           4           2           42.0
137          15          14           65.0
138           6           2           44.0

[139 rows x 3 columns]


In [37]:
train_y = train_df_labelled.loc[:, ['KSSI_total', 'suicidal_idea', 'suicidal_plan', 'suicidal_attempt']]
test_y = test_df_labelled.loc[:, ['KSSI_total', 'suicidal_idea', 'suicidal_plan', 'suicidal_attempt']]
valid_y = valid_df_labelled.loc[:, ['KSSI_total', 'suicidal_idea', 'suicidal_plan', 'suicidal_attempt']]

print(train_y)
print(type(train_y))
print(train_df_labelled['KSSI_total'].dtype)
print(train_df_labelled['suicidal_idea'].dtype)
print(train_df_labelled['suicidal_plan'].dtype)
print(train_df_labelled['suicidal_attempt'].dtype)

train_y = train_y.apply(pd.to_numeric, errors='coerce')
test_y = test_y.apply(pd.to_numeric, errors='coerce')
valid_y = valid_y.apply(pd.to_numeric, errors='coerce')

print(train_y['KSSI_total'].dtype)
print(train_y['suicidal_idea'].dtype)
print(train_y['suicidal_plan'].dtype)
print(train_y['suicidal_attempt'].dtype)

train_y['sum'] = train_y['KSSI_total'] + train_y['suicidal_idea'] + train_y['suicidal_plan'] + train_y['suicidal_attempt']
train_y = train_y.iloc[:, 4]
#train_y = train_y.drop(['KSSI_total', 'suicidal_idea', 'suicidal_plan', 'suicidal_attempt'], axis=1)
print(train_y)
print(type(train_y))

    KSSI_total suicidal_idea suicidal_plan suicidal_attempt
0           12             1             0                0
1            1             0             0                0
2           25             1             0                0
3           50             1             1                1
4            1             0             0                0
..         ...           ...           ...              ...
134          0             0             0                0
135          0             0             0                0
136          1             0             0                0
137         44             1             1                0
138          3             0             0                0

[139 rows x 4 columns]
<class 'pandas.core.frame.DataFrame'>
object
object
object
object
float64
int64
int64
int64
0      13.0
1       1.0
2      26.0
3      53.0
4       1.0
       ... 
134     0.0
135     0.0
136     1.0
137    46.0
138     3.0
Name: sum, Length: 139, dtype: f

In [38]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [39]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [57]:
x_train = torch.from_numpy(train_x.values)
y_train = torch.from_numpy(train_y.values)
torch.nan_to_num_(y_train)
y_train.unsqueeze_(1)

tensor([[ 13.],
        [  1.],
        [ 26.],
        [ 53.],
        [  1.],
        [ 39.],
        [ 54.],
        [  0.],
        [ 70.],
        [ 97.],
        [ 52.],
        [ 77.],
        [ 10.],
        [  5.],
        [ 41.],
        [ 36.],
        [ 27.],
        [107.],
        [ 14.],
        [ 93.],
        [ 23.],
        [ 24.],
        [ 20.],
        [ 38.],
        [ 51.],
        [  6.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  4.],
        [  0.],
        [  0.],
        [  0.],
        [ 13.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  2.],
        [ 45.],
        [ 19.],
        [  4.],
        [ 43.],
        [ 32.],
        [ 67.],
        [ 59.],
        [ 18.],
        [ 83.],
        [ 26.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  5.],
        

In [42]:
from torch.autograd import Variable

class PModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(3, 3)
        self.fc2 = torch.nn.Linear(3, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model = PModel()

In [43]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.005)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0)

In [58]:
nb_epochs =10000
for epoch in range(1, nb_epochs+1):
    prediction = model(x_train.float())
    cost = F.mse_loss(prediction, y_train.float())

    optimizer.zero_grad() 
    cost.backward() 
    optimizer.step() 
    
    if(epoch%100==0):
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

Epoch  100/10000 Cost: 495.519379
Epoch  200/10000 Cost: 495.519379
Epoch  300/10000 Cost: 495.519379
Epoch  400/10000 Cost: 495.519379
Epoch  500/10000 Cost: 495.519379
Epoch  600/10000 Cost: 495.519379
Epoch  700/10000 Cost: 495.519379
Epoch  800/10000 Cost: 495.519379
Epoch  900/10000 Cost: 495.519379
Epoch 1000/10000 Cost: 495.519379
Epoch 1100/10000 Cost: 495.519379
Epoch 1200/10000 Cost: 495.519379
Epoch 1300/10000 Cost: 495.519379
Epoch 1400/10000 Cost: 495.519379
Epoch 1500/10000 Cost: 495.519379
Epoch 1600/10000 Cost: 495.519379
Epoch 1700/10000 Cost: 495.519379
Epoch 1800/10000 Cost: 495.519379
Epoch 1900/10000 Cost: 495.519379
Epoch 2000/10000 Cost: 495.519379
Epoch 2100/10000 Cost: 495.519379
Epoch 2200/10000 Cost: 495.519379
Epoch 2300/10000 Cost: 495.519379
Epoch 2400/10000 Cost: 495.519379
Epoch 2500/10000 Cost: 495.519379
Epoch 2600/10000 Cost: 495.519379
Epoch 2700/10000 Cost: 495.519379
Epoch 2800/10000 Cost: 495.519379
Epoch 2900/10000 Cost: 495.519379
Epoch 3000/100

In [45]:
print(list(model.parameters()))

[Parameter containing:
tensor([[-0.2965, -0.4962, -0.2609],
        [-0.5015, -0.4705, -0.4388],
        [ 0.1099,  0.3225, -0.2975]], requires_grad=True), Parameter containing:
tensor([-0.1074, -0.2664, -0.0285], requires_grad=True), Parameter containing:
tensor([[-0.5020, -0.3275, -0.5362]], requires_grad=True), Parameter containing:
tensor([12.1654], requires_grad=True)]


In [46]:
test_y['sum'] = test_y['KSSI_total'] + test_y['suicidal_idea'] + test_y['suicidal_plan'] + test_y['suicidal_attempt']
test_y = test_y.iloc[:, 4]

In [47]:
x_test = torch.from_numpy(test_x.values)
y_test = torch.from_numpy(test_y.values)
torch.nan_to_num_(y_test)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64)